In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Paths to the folders
source_dir = '/content/drive/MyDrive/Sports/Active/NonSports'  # Adjust to your directory structure

# Data generator with a validation split
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    source_dir,
    target_size=(64, 64),  # Reduced image size for faster computation
    batch_size=16,  # Smaller batch size to process faster
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    source_dir,
    target_size=(64, 64),  # Reduced image size
    batch_size=16,
    class_mode='binary',
    subset='validation'
)

# List of learning rates to test
learning_rates = [0.002, 0.001, 0.01, 0.1]

# Dictionary to store accuracy for each learning rate
accuracy_results = {}

for lr in learning_rates:
    print(f"\nTraining with learning rate: {lr}\n")

    # Load the VGG19 model pre-trained on ImageNet
    vgg19_base = VGG19(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

    # Freeze the convolutional base
    vgg19_base.trainable = False

    # Build the model
    model = models.Sequential([
        vgg19_base,
        layers.Flatten(),
        layers.Dense(64, activation='relu'),  # Reduced Dense layer size
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    # Compile the model with the current learning rate
    model.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics=['accuracy'])

    # Train the model (reduced to 1-2 epochs for fast execution)
    history = model.fit(train_generator, epochs=2, validation_data=validation_generator)

    # Evaluate the model
    test_loss, test_acc = model.evaluate(validation_generator)
    accuracy_results[lr] = test_acc
    print(f"Test accuracy for learning rate {lr}: {test_acc}")

    # Predict on validation data
    Y_pred = model.predict(validation_generator)
    y_pred = np.round(Y_pred).astype(int)

    # Generate confusion matrix and classification report
    cm = confusion_matrix(validation_generator.classes, y_pred)
    cm_labels = list(validation_generator.class_indices.keys())
    print(f"\nConfusion Matrix for learning rate {lr}:")
    print(cm)

    print(f"\nClassification Report for learning rate {lr}:")
    print(classification_report(validation_generator.classes, y_pred, target_names=cm_labels))

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=cm_labels, yticklabels=cm_labels)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title(f'Confusion Matrix (Learning Rate: {lr})')
    plt.show()

    # Generate and print classification report
    report = classification_report(validation_generator.classes, y_pred, target_names=cm_labels, output_dict=True)
    report_df = pd.DataFrame(report).transpose()

    # Display as a table
    print("\nClassification Report as Table:")
    print(report_df)

# Print overall accuracy results for different learning rates
print("\nOverall accuracy results for different learning rates:")
for lr, acc in accuracy_results.items():
    print(f"Learning Rate: {lr} - Accuracy: {acc}")

# Optionally, plot the accuracy results for different learning rates
plt.figure(figsize=(8, 4))
plt.plot(list(accuracy_results.keys()), list(accuracy_results.values()), marker='o')
plt.title('Accuracy vs Learning Rate')
plt.xlabel('Learning Rate')
plt.ylabel('Accuracy')
plt.xscale('log')
plt.show()